# 🎮 Pon# 🎮 Pong

TODO: Texto explicando o problema

TODO: Explicar Numpy (?)

In [ ]:
# Importando a biblioteca Numpy
import numpy as np

TODO: Explicar variáveis

In [ ]:
# Constantes da Política Epsilon Greedy
# Epsilon: probabilidade de experimentar uma ação aleatória
EPSILON = 0.7        # Valor inicial do epsilon
EPSILON_MIN = 0.01   # Valor mínimo de epsilon
DECAIMENTO = 0.98    # Fator de decaímento do epsilon (por episódio)

# Hiperparâmetros do Q-Learning
ALFA = 0.05          # Learning rate
GAMA = 0.9           # Fator de desconto

N_EPISODIOS = 250    # Quantidade de episódios que treinaremos

# Dicionário dos valores de Q
# Chaves: estados; valores: qualidade Q atribuida a cada ação
Q = {}

TODO: Explicar Ambiente

In [ ]:
# Importando a Biblioteca Gym
import gym

# Criando o nosso Ambiente: Pong
env = gym.make("pong:turing-easy-v0")

# Número total de ações: 3
# 0 = parado; 1 = baixo; 2 = cima
n_acoes = env.action_space.n

print('Número de ações:', n_acoes)

TODO: Explicar Discretização

In [ ]:
def discretiza_estado(estado):
    return tuple(round(x/10) for x in estado)

TODO: Explicar tomada de ação

In [ ]:
def escolhe_acao(env, Q, estado, epsilon):
    # Se não conhecermos ainda o estado, inicializamos o Q de cada ação como 0
    if estado not in Q.keys(): Q[estado] = [0] * n_acoes

    # Escolhemos um número aleatório com "np.random.random()"
    # Se esse número for menor que epsilon, tomamos uma ação aleatória
    if np.random.random() < epsilon:
        # Escolhemos uma ação aleatória, com env.action_space.sample()
        acao = ...
    else:
        # Escolhemos a melhor ação para o estado atual, com np.argmax()
        acao = ...
    return acao

TODO: Explicar função de rodar partida

In [ ]:
def roda_partida(env, Q, renderiza=True):
    # Resetamos o ambiente
    estado = env.reset()

    # Discretizamos o estado
    estado = ...
    
    done = False
    retorno = 0
    
    while not done:
        # Escolhemos uma ação
        acao = ...

        # Tomamos nossa ação escolhida e recebemos informações do próximo estado
        prox_estado, recompensa, done, info = ...

        # Discretizamos o próximo estado
        prox_estado = ...

        # Renderizamos o Ambiente
        if renderiza:
            env.render()

        retorno += recompensa
        estado = prox_estado

    print(f'retorno {retorno:.1f},  '
          f'placar {env.score[0]}x{env.score[1]}')
    
    env.close()

In [ ]:
# Rodamos uma partida de Pon
roda_partida(env, Q)

## 🏋️‍♀️ Treinamento

TODO: Falar de Treinamento

TODO: Explicar Bellman (o básico, já foi abordado antes)

In [ ]:
def atualiza_q(Q, estado, acao, recompensa, prox_estado):
    # para cada estado ainda não descoberto, iniciamos seu valor como nulo
    if estado not in Q.keys(): Q[estado] = [0] * n_acoes
    if prox_estado not in Q.keys(): Q[prox_estado] = [0] * n_acoes

    # equação de Bellman
    Q[estado][acao] = ...

TODO: Explicar (brevemente?) pickle

In [ ]:
import pickle

def salva_tabela(Q, nome = 'model.pickle'):
    with open(nome, 'wb') as pickle_out:
        pickle.dump(Q, pickle_out)

def carrega_tabela(nome = 'model.pickle'):
    with open(nome, 'rb') as pickle_out:
        return pickle.load(pickle_out)

TODO: Explicar função de treinamento

In [ ]:
def treina(env, Q):
    retornos = []      # retorno de cada episódio
    epsilon = EPSILON

    for episodio in range(1, N_EPISODIOS+1):
        # resetar o ambiente e discretizar a ação
        ...
        
        done = False
        retorno = 0
        
        while not done:
            # escolher uma ação
            ...

            # tomar a ação
            ...

            # discretizar o próximo estado
            ...

            atualiza_q(Q, estado, acao, recompensa, prox_estado)

            retorno += recompensa
            estado = prox_estado

        # calcular o próximo epsilon
        epsilon = ...
        epsilon = max(epsilon, EPSILON_MIN)

        retornos.append(retorno)

        if episodio % 10 == 0:
            salva_tabela(Q)

        print(f'episódio {episodio},  '
              f'retorno {retorno:7.1f},  '
              f'retorno médio (últimos 10 episódios) {np.mean(retornos[-10:]):7.1f},  '
              f'placar {env.score[0]}x{env.score[1]},  '
              f'epsilon: {epsilon:.3f}')
        
    env.close()

In [ ]:
treina(env, Q)

## 🏓 Testando nosso Agente Treinado

In [ ]:
roda_partida(env, Q)